In [ ]:
!pip install transformers
!pip install --upgrade transformers

In [ ]:
!pip install Flask pyngrok


In [ ]:
!ngrok authtoken #urtoken



In [ ]:
!pip install flask-cors


In [ ]:

from flask import Flask, request, jsonify  # Flask imports for web framework and request/response handling
from transformers import BlenderbotForConditionalGeneration, BlenderbotTokenizer  # For BlenderBot model and tokenizer
from pyngrok import ngrok  # For exposing the Flask app via ngrok
import torch  # For handling PyTorch tensors and models
from flask_cors import CORS  # To enable Cross-Origin Resource Sharing

# Initialize Flask app
app = Flask(__name__)
CORS(app)  # Enable CORS for the Flask app

# Load the fine-tuned Blenderbot model and tokenizer
model = BlenderbotForConditionalGeneration.from_pretrained('SomeUser675/mentalconvobot')
tokenizer = BlenderbotTokenizer.from_pretrained('SomeUser675/mentalconvobot')

# Function to generate a response using BlenderBot
def generate_response(input_text, max_length=150, min_length=30, temperature=0.7, top_k=50, top_p=0.95):
    # Tokenize the input text
    inputs = tokenizer(input_text, return_tensors="pt")
    inputs = {key: value.to(model.device) for key, value in inputs.items()}

    # Generate a response without gradients
    with torch.no_grad():
        response_ids = model.generate(
            inputs['input_ids'],
            max_length=max_length,
            min_length=min_length,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            num_beams=3,
            do_sample=True,
            length_penalty=1.0,
            early_stopping=True
        )
        # Decode the generated response tokens into a string
        response = tokenizer.decode(response_ids[0], skip_special_tokens=True)

    return response

# Define a route for the chatbot interaction
@app.route('/chat', methods=['POST'])
def chat():
    input_text = request.json.get('message')  # Get the message from the POST request
    if not input_text:
        return jsonify({'error': 'No input text provided'}), 400

    # Generate a response and send it back as JSON
    response = generate_response(input_text)
    return jsonify({'response': response})

# Health check route to ensure the server is running
@app.route('/')
def health_check():
    return "Chatbot is running!"

# Start the Flask app and expose it via ngrok
if __name__ == '__main__':
    # Set up ngrok to expose the local server
    url = ngrok.connect(5000)
    print(f"Public URL: {url}")
    # Run the Flask app on port 5000
    app.run(host='0.0.0.0', port=5000)
